In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/kaggle/input/cvprojectdataset/myData"
img_size = (32, 32)
batch_size = 64

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,        # 80/20 train/val
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

2025-10-19 20:33:32.465872: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760906012.880293      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760906012.989671      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 128128 images belonging to 43 classes.
Found 32000 images belonging to 43 classes.


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

num_classes = 43
img_size = (32, 32)

model = models.Sequential([
    layers.Input(shape=(*img_size, 3)),          # RGB
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Conv2D(64, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

model.summary()

I0000 00:00:1760906160.081109      37 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1760906160.081817      37 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 10, 10, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       409,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 43)             │        11,051 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 486,475 (1.86 MB)

 Trainable params: 486,475 (1.86 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [5]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

ckpt_path = "/kaggle/working/traffic_signs_cnn.keras"

cbs = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5, verbose=1),
    ModelCheckpoint(ckpt_path, monitor='val_accuracy', save_best_only=True, verbose=1)
]

In [6]:
history = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    callbacks=cbs,
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1760906178.600822      98 service.cc:148] XLA service 0x7a6154003f60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1760906178.602633      98 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1760906178.602653      98 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1760906179.051458      98 cuda_dnn.cc:529] Loaded cuDNN version 90300


   1/2002 ━━━━━━━━━━━━━━━━━━━━ 4:43:55 9s/step - accuracy: 0.0312 - loss: 3.7630

I0000 00:00:1760906183.844459      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2002/2002 ━━━━━━━━━━━━━━━━━━━━ 0s 547ms/step - accuracy: 0.4043 - loss: 2.1527
Epoch 1: val_accuracy improved from -inf to 0.93144, saving model to /kaggle/working/traffic_signs_cnn.keras
2002/2002 ━━━━━━━━━━━━━━━━━━━━ 1388s 689ms/step - accuracy: 0.4044 - loss: 2.1522 - val_accuracy: 0.9314 - val_loss: 0.2517 - learning_rate: 0.0010
Epoch 2/10
2002/2002 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - accuracy: 0.8693 - loss: 0.4196
Epoch 2: val_accuracy improved from 0.93144 to 0.96859, saving model to /kaggle/working/traffic_signs_cnn.keras
2002/2002 ━━━━━━━━━━━━━━━━━━━━ 490s 245ms/step - accuracy: 0.8693 - loss: 0.4196 - val_accuracy: 0.9686 - val_loss: 0.1081 - learning_rate: 0.0010
Epoch 3/10
2002/2002 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - accuracy: 0.9225 - loss: 0.2573
Epoch 3: val_accuracy improved from 0.96859 to 0.98019, saving model to /kaggle/working/traffic_signs_cnn.keras
2002/2002 ━━━━━━━━━━━━━━━━━━━━ 453s 226ms/step - accuracy: 0.9225 - loss: 0.2573 - val_accuracy: 0.9802 - val_los

In [9]:
val_loss, val_acc = model.evaluate(val_gen, verbose=0)
print(f"Val accuracy: {val_acc*100:.2f}%")
model.save("/kaggle/working/traffic_signs_cnn_final.keras")

Val accuracy: 99.19%
